## TiGa - ProHack Notebook

## Aproach
1. Define the target for prediction - Y (index)
2. Import Libraries
3. Import Data

### Step 1: Import Libraries

In [ ]:
# for working with dataframes import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from scipy.optimize import minimize

import warnings 
warnings.filterwarnings('ignore')

### Step 2: Import data

In [ ]:
# train data
train_df = pd.read_csv('train.csv')

train_df.head()

### Step 3: Analyze & Pre-processing Data

In [ ]:
train_df.dtypes

In [ ]:
train_df.describe(include="all")

#### Evaluate for missing data

The output is a boolean value indicating whether the value that is passed into the argument is in fact missing data.

In [ ]:
missing_data = train_df.isnull()
missing_data.head(5)

<h4>Count missing values in each column</h4>
<p>
Using a for loop in Python, we can quickly figure out the number of missing values in each column. As mentioned above, "True" represents a missing value, "False"  means the value is present in the dataset.  In the body of the for loop the method  ".value_counts()"  counts the number of "True" values. 
</p>

In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

<h3 id="deal_missing_values">Deal with missing data</h3>
<b>How to deal with missing data?</b>

<ol>
    <li>drop data<br>
        a. drop the whole row<br>
        b. drop the whole column
    </li>
    <li>replace data<br>
        a. replace it by mean<br>
        b. replace it by frequency<br>
        c. replace it based on other functions
    </li>
</ol>

<i> !!! Whole columns should be dropped only if most entries in the column are empty. !!! </i>  

#### Dropping data
Drop data columns based on the relativity to the well-being index. The selection mostly follows the OECD approach with their index.

<b> Replace by mean</b>
<ul>
    <li> existence expectancy index
    <li> existence expectancy at birth
    <li> Gross income per capita
    <li> Income Index
    <li>
</ul>

In [ ]:
# calculate the average
avg_exist_exp_i = train_df["existence expectancy index"].astype("float").mean(axis=0)
print("Average of existence expectancy index:", avg_exist_exp_i)

In [ ]:
avg_exist_exp_at_b = train_df["existence expectancy at birth"].astype("float").mean(axis=0)
print("Average of existence expectancy at birth:", avg_exist_exp_at_b)

In [ ]:
#Gross income per capita

avg_gross_inc_pc = train_df["Gross income per capita"].astype("float").mean(axis=0)
print("Average of Gross income per capita:", avg_gross_inc_pc)

In [ ]:
#Income Index

avg_income_i = train_df["Income Index"].astype("float").mean(axis=0)
print("Average of Income Index:", avg_income_i)

#### Replacing data by mean

In [ ]:
train_df["existence expectancy index"].replace(np.nan, avg_exist_exp_i, inplace=True)

In [ ]:
train_df["existence expectancy at birth"].replace(np.nan, avg_exist_exp_at_b, inplace=True)

In [ ]:
#Gross income per capita
train_df["Gross income per capita"].replace(np.nan, avg_gross_inc_pc, inplace=True)

In [ ]:
#Income Index
train_df["Income Index"].replace(np.nan, avg_income_i, inplace=True)

In [ ]:
#check
missing_data = train_df.isnull()

print (missing_data["existence expectancy index"].value_counts())

#### Drop rows

In [ ]:
train1=train_df.drop(train_df.index[363:902])

In [ ]:
train2=train1.drop(train1.index[543:1263])

In [ ]:
train3=train2.drop(train2.index[722:1441])

In [ ]:
train4=train3.drop(train3.index[902:1351])

In [ ]:
# after rows dropping here is the shape of an updated dataframe

train4.shape

In [ ]:
train_df.shape

In [ ]:
missing_data1 = train4.isnull()

for column in missing_data1.columns.values.tolist():
    print(column)
    print (missing_data1[column].value_counts())
    print("")

### Counting galaxies

In [ ]:
train_gal=set(train4["galaxy"])
s=0
for x in train_gal:
    s=s+len(train4.loc[train4['galaxy'] == x])
print("Total distinct galaxies: {}".format(len(train_gal)))
print("Average samples per galaxy: {}".format(s/len(train_gal)))

### Methods for Cross-validating Training Data

#### Approach by Ömer Gözüaçık

I trained a model for exery distinct galaxy in the training set (180) except the one from 126th galaxy as it has only one sample.

I used features with top x correlation with respect to y (target variable) galaxy specific. (x is found by trying different values [20,25,30,40,50,60,70])

Missing values are filled with the galaxy specific 'mean' of the data. (Median can be used alternatively.)

Train and test sets are not mixed for both imputation and standardization.

Standard Scaler is used to standardize data.

Gradient Boosted Regression is used as a model.

In [ ]:
def cross_validation_loop(data,cor):
    labels= data['y']
    data=data.drop('galaxy', axis=1)    
    data=data.drop('y', axis=1)
    
    correlation=abs(data.corrwith(labels))
    columns=correlation.nlargest(cor).index
    data=data[columns]
    
    imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(data)
    data=imp.transform(data)

    scaler = StandardScaler().fit(data)
    data = scaler.transform(data)
        
    estimator = GradientBoostingRegressor(n_estimators=300)
    
    cv_results = cross_validate(estimator, data, labels, cv=4, scoring='neg_root_mean_squared_error')

    error=np.mean(cv_results['test_score'])
    
    return error

#### Code for cross-validating a model for every galaxy.

I return the mean of the cross-validation scores disregarding the differences of their sample sizes.
Remove the lonely galaxy, occuring only once.

In [ ]:
train_gal=set(train4["galaxy"])
train_gal.remove("NGC 5253")
def loop_train(cor):
    errors=[]
    for gal in train_gal:
        index = train4.index[train4['galaxy'] == gal]
        data = train4.loc[index]
        errors.append(cross_validation_loop(data,cor))
    return np.mean(errors)

#### Checking which correlation threshold gives better value
The model performs best when the threshold is 20 with RMSE of 0.0063

In [ ]:
cor=[20,25,30,40,50,60,70,80]
errors=[]
for x in cor: 
    errors.append(loop_train(x))

In [ ]:
print(errors)

## Making predictions on the test data

- Similar methodology is used to fill the missing value and standardization.
- The best covariance threshold in the cross validation, 20, is used.

In [ ]:
def test_loop(data, test_data):
    labels= data['y']
    data=data.drop('galaxy', axis=1)    
    data=data.drop('y', axis=1)
    correlation=abs(data.corrwith(labels))
    columns=correlation.nlargest(20).index
    
    train_labels= labels
    train_data=data[columns]
    test_data= test_data[columns]
    
    imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(train_data)
    train_data=imp.transform(train_data)
    test_data=imp.transform(test_data)

    scaler = StandardScaler().fit(train_data)
    train_data = scaler.transform(train_data)
    test_data = scaler.transform(test_data)

    model = GradientBoostingRegressor(n_estimators=300)
    model.fit(train_data, train_labels)

    predictions = model.predict(test_data)
    return predictions

#### Sorting samples with respect to their unique galaxy type. 


In [ ]:
test=test_res
test=test.sort_values(by=['galaxy'])
test_pred = pd.DataFrame(0, index=np.arange(len(test)), columns=["predicted_y"])

#### Looping over all galaxy types in the test set and making predictions.

In [ ]:
i=0
for gal in test_gal:
    count=len(test.loc[test['galaxy'] == gal])
    index = train.index[train['galaxy'] == gal]
    data = train.loc[index]
    pred=test_loop(data,test.loc[test['galaxy']==gal])
    test_pred.loc[i:i+count-1,'predicted_y'] = pred
    i=i+count 

#### Sorting samples with respect to the index.

In [ ]:
test["predicted_y"]=test_pred.to_numpy()
test.sort_index(inplace=True)
predictions = test["predicted_y"]

## Discussion 1

- With this approach, we are **not using 8 galaxies in the training set as they are not in the test set.** (Almost 160 samples)

- A better approach should use them as well.

- According to our theory, every galaxy represent a country and samples are its properties at a time (maybe galactic year represents time).

- Some countries may have missing values as they may have joined IBRD late. This may be organizers decision as well. Filling missing values with regression can improve performance.

- World Bank categorizes countries by both region and income: https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups

7 regions: East Asia and Pacific, Europe and Central Asia, Latin America & the Caribbean, Middle East and North Africa, North America, South Asia, Sub-Saharan Africa

4 income groups: Low-income economies, Lower-middle-income economies, Upper-middle-income economies, High-income economies 

- Clustering galaxies may excel the performance of the model. I would try both clustering galaxies to either 4 or 7 clusters. Then try making imputation/training with respect to every cluster.

This code is a summary of what we have done. We also analyzed RMSE for cross-validation for per galaxy. 

Galaxies: {128, 2, 4, 5, 133, 11, 140, 147, 153, 154, 34, 35, 40, 43, 55, 64, 76, 78, 83, 100, 101, 102, 107, 108, 119} have RMSE over 0.008. 

The list gives them in order, 128th having 0.008559 and 119th having 0.034926. 

- Fine tuning these problematic galaxies with low cross-validation scores can excel the performance of the model

## Optimization part

- Ideally giving 100 to top 500 samples with highest p^2 values should optimize the likely increase.
- However, as the predictions can be faulty, this approach would result with lower Leaderboard Score.

E.g: If the original p^2 value is higher than the predicted p^2, it will increase the error as we are directly giving it 0.

- That's why, I believe its better to spread the risk for the samples in the bordering regions (400< [rank of p^2] <600).
- I assign 100 energy to top 400 samples and 50 energy to the remaining top 200 samples.

In [ ]:
index = predictions
pot_inc = -np.log(index+0.01)+3

In [ ]:
p2= pot_inc**2

In [ ]:
ss = pd.DataFrame({
    'Index':test.index,
    'pred': predictions,
    'opt_pred':0,
    'eei':test['existence expectancy index'], # So we can split into low and high EEI galaxies
})

In [ ]:
ss.loc[p2.nlargest(400).index, 'opt_pred']=100
ss=ss.sort_values('pred')
ss.iloc[400:600].opt_pred = 50
ss=ss.sort_index()

In [ ]:
increase = (ss['opt_pred']*p2)/1000

In [ ]:
print(sum(increase), ss.loc[ss.eei < 0.7, 'opt_pred'].sum(), ss['opt_pred'].sum())

In [ ]:
ss[['Index', 'pred', 'opt_pred']].to_csv('submission3.csv', index=False)

## Discussion 2

- Optimization can be done better by changing the spreading the risk part (assigning energy to the 400<p^2<600 region).

- You can give values that are decreasing starting from 400th to 600th (99, 98, 97...). 

It is less likely for the 400th sample to be out of top 500, and similarly it is less likely for the 600th sample to be in the top 500. That's why, you can give more energy to the ones in the 400-500 region and less to the 500-600.

- This approach got me and my friend to the best score of 0.04271993 which is ranked 22nd right now.

- As we are out of top 20 and reached the upload limit, we are sharing our approach publicly to help other teams that have worse results. 
